# Gran Analysis: This script is designed to analyze alkalinity titration data. 

## Define a custom function
This definition helps us find the point where we want to begin the Gran analysis below.

In [ ]:
# a definition called getIndexes to find the position of a value in a dataframe
def find_neighbours(df, value):
    exactmatch = df[df["Volume (ml)"] == value]
    if not exactmatch.empty:
        return exactmatch.index
    else:
        lowerneighbour_ind = df[df["Volume (ml)"] < value]["Volume (ml)"].idxmax()
        upperneighbour_ind = df[df["Volume (ml)"] > value]["Volume (ml)"].idxmin()
        return [lowerneighbour_ind, upperneighbour_ind]


# Data input
Replace `alk7.csv` with the filename of your data and input your conductivity values.

In [ ]:
# This cell puts the uploaded csv file into a pandas dataframe
# The file name needs to match exactly 
import pandas as pd
# Replace the file name with the name of sample file
df1 = pd.read_csv("alk7.csv")
# Replace the value with the conductivity of your a sample 
cond_a = 130.0
# Replace the file name with the name of sample file
df2 = pd.read_csv("alk7.csv")
# Replace the value with the conductivity of your b sample
cond_b = 130.0


# Examine the data of sample a (Mercer Slough)
This is your raw titration data. You might notice burette refilling artifacts since the burette can only hold 1 ml of acid. This figure is saved for you to include in your lab report.

In [ ]:
# to plot the data we need to import matlibplot
import matplotlib.pyplot as plt

# to make the plots look nicer we will import the seaborn module
import seaborn as sns

sns.set_theme()
# plot the data and label the axes
plt.scatter(df1["pH"], df1["Volume (ml)"])
plt.xlabel("pH")
plt.ylabel("Volume (ml)")
plt.show()


# Gran Analysis of sample a (Mercer Slough)
The Gran function is calculated and plotted for you.

In [ ]:
# import a plotting library that can be used to make interactive plots
import plotly.express as px

# calculate the Gran function for the entire data set assuming an activity coefficient of 1
# volume of sample
Vs = df1["sampvol"][0]
# normality of the acid
Nt = 0.076
# H+ activity
H = 10 ** -df1["pH"].values
# Ionic strength from conductivity
I = 1.9e-5 * cond_a
# activity coefficient
log_gamma = -0.5085*1**2*((I**0.5/(1+I**0.5)) - 0.3*I)
act_coef = 10**log_gamma
# calculate gran function
F1 = ((Vs + df1["Volume (ml)"].values) / Vs) * (H / act_coef)

# plot the Gran function
fig = px.scatter(df1, x="Volume (ml)", y=F1)
fig.show()


# Examine the linear portion of the Gran function and indicate the volume where the linear portion of the Gran function begins
To determine the end point for the titration, we only need to use data at the end of the titration where the data are linear and increasing rapidly. Note the volume where the data start to increase rapidly.

In [ ]:
# asks for the volume where the linear portion of the Gran function begins and finds the index value for that point
startvol = input(
    "Examine the Gran plot and input the volume you would like to start the regression for the linear portion of the plot: "
)
ind = find_neighbours(df1, float(startvol))


# Examine the extract Gran function
Select the next cell and run it. Take a look at the plot and see if you've selected the linear portion of the data. If you need to refine your start point, run the cell above again and revise the start volume. Run the cell below again to check your refinement.

In [ ]:
# define variables with the extract of data we are interested in
vol = df1["Volume (ml)"][ind[0] : -1].values
F1ex = F1[ind[0] : -1]
# I'll plot just that section of the Gran function
plt.figure()
plt.plot(vol, F1ex, marker="x", linestyle="none")
plt.xlabel("Volume (ml)")
plt.ylabel("F1")
plt.show()


# Final analysis of sample a (Mercer Slough)
To determine the endpoint for our titration we need to fit a curve to this section of the gran function and then extrapolate that line back to the x-axis. The point where our extrapolated line intersects the x-axis is the endpoint volume for our titration. We'll use the `numpy` module to help us do that. The final plot is saved for you to include in your lab report.

In [ ]:
# import the numpy module so we can call various functions for the linear regression
import numpy as np

# get the coefficients for the linear regression fit to our extracted portion of data
coefficients = np.polyfit(vol, F1ex, 1)

# put those coefficients into a 1st degree polynomial so we can calculate some points to
# show the linear regression on our plot
polynomial = np.poly1d(coefficients)

# choose a couple of x values to calculate y values for plotting our linear regression
volex = [df1["Volume (ml)"][ind[0]] - 0.05, df1["Volume (ml)"].iloc[-1]]
fit_line = polynomial(volex)

# calculate the end point of the titration (i.e., the x value when y =0)
endpoint = round(-1 * coefficients[1] / coefficients[0], 3)

# calculate the alkalinity using the end point vol, sample volume, and acid concentration
alk = Nt * endpoint / Vs
print("[alk] = %.3e M" % alk)
print("Approximate ionic strength = %.4f M" % I)
print("Approximate activity coefficient of H+ = %.2f" % act_coef)
# plot the extracted portion of the Gran function, the linear regression, and end point
plt.figure()
plt.plot(vol, F1ex, marker="x", linestyle="none", label="Gran Function")
plt.plot(volex, fit_line, label="linear regression")
plt.scatter(endpoint, 0, marker="o", label="end point")
plt.xlabel("Volume (ml)")
plt.ylabel("F1")
plt.legend()

# annotate the plot to show the value of the end point
plt.annotate(
    str(endpoint),
    xy=(endpoint, 0),
    xycoords="data",
    xytext=(0.26, 0.5),
    textcoords="axes fraction",
    arrowprops=dict(facecolor="black", shrink=0.05),
    horizontalalignment="center",
    verticalalignment="center",
)

# Save and download the plot
plt.savefig("alk_analysis_r1.png", bbox_inches="tight")
plt.savefig("alk_analysis_r1.pdf", bbox_inches="tight")


# Examine the data of sample b
This is your raw titration data. You might notice burette refilling artifacts since the burette can only hold 1 ml of acid. This figure is saved for you to include in your lab report.

In [ ]:
# to plot the data we need to import matlibplot
import matplotlib.pyplot as plt

# to make the plots look nicer we will import the seaborn module
import seaborn as sns

sns.set_theme()
# plot the data and label the axes
plt.scatter(df2["pH"], df2["Volume (ml)"])
plt.xlabel("pH")
plt.ylabel("Volume (ml)")
plt.show()


# Gran Analysis of sample b
The Gran function is calculated and plotted for you.

In [ ]:
# import a plotting library that can be used to make interactive plots
import plotly.express as px

# calculate the Gran function for the entire data set assuming an activity coefficient of 1
# volume of sample
Vs = df2["sampvol"][0]
# normality of the acid
Nt = 0.076
# H+ activity
H = 10 ** -df2["pH"].values
# Ionic strength from conductivity
I = 1.9e-5 * cond_b
# activity coefficient
log_gamma = -0.5085*1**2*((I**0.5/(1+I**0.5)) - 0.3*I)
act_coef = 10**log_gamma
# calculate gran function
F1 = ((Vs + df2["Volume (ml)"].values) / Vs) * (H / act_coef)

# plot the Gran function
fig = px.scatter(df2, x="Volume (ml)", y=F1)
fig.show()


# Examine the linear portion of the Gran function and indicate the volume where the linear portion of the Gran function begins
To determine the end point for the titration, we only need to use data at the end of the titration where the data are linear and increasing rapidly. Note the volume where the data start to increase rapidly.

In [ ]:
# asks for the volume where the linear portion of the Gran function begins and finds the index value for that point
startvol = input(
    "Examine the Gran plot and input the volume you would like to start the regression for the linear portion of the plot: "
)
ind = find_neighbours(df2, float(startvol))


# Examine the extract Gran function
Select the next cell and run it. Take a look at the plot and see if you've selected the linear portion of the data. If you need to refine your start point, run the cell above again and revise the start volume. Run the cell below again to check your refinement.

In [ ]:
# define variables with the extract of data we are interested in
vol = df2["Volume (ml)"][ind[0] : -1].values
F1ex = F1[ind[0] : -1]
# I'll plot just that section of the Gran function
plt.figure()
plt.plot(vol, F1ex, marker="x", linestyle="none")
plt.xlabel("Volume (ml)")
plt.ylabel("F1")
plt.show()


# Final analysis of sample b
To determine the endpoint for our titration we need to fit a curve to this section of the gran function and then extrapolate that line back to the x-axis. The point where our extrapolated line intersects the x-axis is the endpoint volume for our titration. We'll use the `numpy` module to help us do that. The final plot is saved for you to include in your lab report.

In [ ]:
# import the numpy module so we can call various functions for the linear regression
import numpy as np

# get the coefficients for the linear regression fit to our extracted portion of data
coefficients = np.polyfit(vol, F1ex, 1)

# put those coefficients into a 1st degree polynomial so we can calculate some points to
# show the linear regression on our plot
polynomial = np.poly1d(coefficients)

# choose a couple of x values to calculate y values for plotting our linear regression
volex = [df2["Volume (ml)"][ind[0]] - 0.05, df2["Volume (ml)"].iloc[-1]]
fit_line = polynomial(volex)

# calculate the end point of the titration (i.e., the x value when y =0)
endpoint = round(-1 * coefficients[1] / coefficients[0], 3)

# calculate the alkalinity using the end point vol, sample volume, and acid concentration
alk = Nt * endpoint / Vs
print("[alk] = %.3e M" % alk)
print("Approximate ionic strength = %.4f M" % I)
print("Approximate activity coefficient of H+ = %.2f" % act_coef)
# plot the extracted portion of the Gran function, the linear regression, and end point
plt.figure()
plt.plot(vol, F1ex, marker="x", linestyle="none", label="Gran Function")
plt.plot(volex, fit_line, label="linear regression")
plt.scatter(endpoint, 0, marker="o", label="end point")
plt.xlabel("Volume (ml)")
plt.ylabel("F1")
plt.legend()

# annotate the plot to show the value of the end point
plt.annotate(
    str(endpoint),
    xy=(endpoint, 0),
    xycoords="data",
    xytext=(0.26, 0.5),
    textcoords="axes fraction",
    arrowprops=dict(facecolor="black", shrink=0.05),
    horizontalalignment="center",
    verticalalignment="center",
)

# Save and download the plot
plt.savefig("alk_analysis_r2.png", bbox_inches="tight")
plt.savefig("alk_analysis_r2.pdf", bbox_inches="tight")


# Raw titration figure for report

In [ ]:
plt.plot(df1["pH"], df1["Volume (ml)"], label="Mercer Slough")
plt.plot(df2["pH"], df2["Volume (ml)"], label="sample b", linestyle="--")
plt.xlabel("pH")
plt.ylabel("Volume (ml)")
plt.legend()
plt.savefig("raw_titration.png", bbox_inches="tight")
plt.savefig("raw_titration.pdf", bbox_inches="tight")
